In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path
import fileinput

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)

def replace(file,pattern,replace):
    myfile = fileinput.FileInput(file, inplace=True)
    for line in myfile:
        line = re.sub(pattern,
                  replace,
                  line)
        print(line)
        
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [2]:
instanceName = 'gotm-cal'
username = 'jose-luis'

createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family debian-9 \
--image-project debian-cloud \
--machine-type n1-standard-2 \
'''
#--boot-disk-size 200GB \

deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/gotmKeys')

if (platform.system() == 'Windows'):
    keyDir = ('c:\\Users\\Guerrero\\Envs\\prognos_calibration\\prognos_calibration\\gotmKeys')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,r"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

Instance gotm-cal is 35.198.160.181
The ip of gotm-cal is 35.198.160.181


## Updating fabfile.py with credentials and ip

In [3]:
if (platform.system() == "Windows"):
    callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
    callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
    callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
    callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

if (platform.system() == "Windows"):    
    replace("fabfile.py", r"^env\.hosts.*",         "env.hosts=['{}']".format('dummy'))
    replace("fabfile.py", r"^env\.user.*",          "env.user='{}'".format(username))
    replace("fabfile.py", r"^env\.key_filename.*",  "env.key_filename='{}'".format(os.path.join(keyDir,username)))
    replace("fabfile.py", r"^env\.roledefs.*",      "env.roledefs={{'{}':['{}'],".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

(b'Linux gotm-cal 4.9.0-7-amd64 #1 SMP Debian 4.9.110-3+deb9u1 (2018-08-03) x86_64\n\nThe programs included with the Debian GNU/Linux system are free software;\nthe exact distribution terms for each program are described in the\nindividual files in /usr/share/doc/*/copyright.\n\nDebian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent\npermitted by applicable law.\n', None)


## Setting up GOTM and FABM in the remote machine

In [4]:
callPopen('fab testConnection')
callPopen('fab getUtilities')
callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
callPopen('fab compileModels')
callPopen('fab getPROGNOS')
callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))

fab testConnection
[dummy] Executing task 'testConnection'
[35.198.160.181] Executing task 'whoAmI'
[35.198.160.181] run: uname -a
[35.198.160.181] out: Linux gotm-cal 4.9.0-7-amd64 #1 SMP Debian 4.9.110-3+deb9u1 (2018-08-03) x86_64 GNU/Linux
[35.198.160.181] out: 

[35.198.160.181] run: whoami
[35.198.160.181] out: jose-luis
[35.198.160.181] out: 


Done.
Disconnecting from 35.198.160.181... done.
fab getUtilities
[dummy] Executing task 'getUtilities'
[dummy] Executing task 'update'
[35.198.160.181] Executing task 'updateMachine'
[35.198.160.181] run: sudo apt-get update
[35.198.160.181] out: 
[35.198.160.181] out: 0% [Working]
[35.198.160.181] out:             
[35.198.160.181] out: Ign:1 http://deb.debian.org/debian stretch InRelease
[35.198.160.181] out: 
[35.198.160.181] out: 0% [Waiting for headers] [Connecting to prod.debian.map.fastly.net]
[35.198.160.181] out:                                                                    
[35.198.160.181] out: Hit:2 http://security.debian

[35.198.160.181] out: python-pip is already the newest version (9.0.1-2).
[35.198.160.181] out: 0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
[35.198.160.181] out: 

[35.198.160.181] Executing task 'getModules'
[35.198.160.181] run: sudo pip install xmlstore editscenario xmlplot matplotlib
[35.198.160.181] out: Requirement already satisfied: xmlstore in /usr/local/lib/python2.7/dist-packages
[35.198.160.181] out: Requirement already satisfied: editscenario in /usr/local/lib/python2.7/dist-packages
[35.198.160.181] out: Requirement already satisfied: xmlplot in /usr/local/lib/python2.7/dist-packages
[35.198.160.181] out: Requirement already satisfied: matplotlib in /usr/local/lib/python2.7/dist-packages
[35.198.160.181] out: Requirement already satisfied: numpy in /usr/local/lib/python2.7/dist-packages (from xmlplot)
[35.198.160.181] out: Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python2.7/dist-packages (from matplotlib)
[35.198.160.181] out: Require

[35.198.160.181] out: [ 33%] Built target fabm_models_hzg
[35.198.160.181] out: Scanning dependencies of target fabm_models_iow
[35.198.160.181] out: [ 34%] Building Fortran object models/iow/CMakeFiles/fabm_models_iow.dir/age/iow_age.F90.o
[35.198.160.181] out: [ 35%] Building Fortran object models/iow/CMakeFiles/fabm_models_iow.dir/ergom/ergom_cgt.F90.o
[35.198.160.181] out: [ 36%] Building Fortran object models/iow/CMakeFiles/fabm_models_iow.dir/spm/spm.F90.o
[35.198.160.181] out: [ 37%] Building Fortran object models/iow/CMakeFiles/fabm_models_iow.dir/iow_model_library.F90.o
[35.198.160.181] out: [ 37%] Built target fabm_models_iow
[35.198.160.181] out: Scanning dependencies of target fabm_models_klimacampus
[35.198.160.181] out: [ 38%] Building Fortran object models/klimacampus/CMakeFiles/fabm_models_klimacampus.dir/phy_feedback/phy_feedback.F90.o
[35.198.160.181] out: [ 38%] Built target fabm_models_klimacampus
[35.198.160.181] out: Scanning dependencies of target fabm_models_met

[35.198.160.181] out: [ 90%] Building Fortran object models/niva/CMakeFiles/fabm_models_niva.dir/niva_model_library.F90.o
[35.198.160.181] out: [ 90%] Built target fabm_models_niva
[35.198.160.181] out: Scanning dependencies of target fabm_models_akvaplan
[35.198.160.181] out: [ 91%] Building Fortran object models/akvaplan/CMakeFiles/fabm_models_akvaplan.dir/plume_injection.F90.o
[35.198.160.181] out: [ 92%] Building Fortran object models/akvaplan/CMakeFiles/fabm_models_akvaplan.dir/tracer.F90.o
[35.198.160.181] out: [ 93%] Building Fortran object models/akvaplan/CMakeFiles/fabm_models_akvaplan.dir/akvaplan_model_library.F90.o
[35.198.160.181] out: [ 93%] Built target fabm_models_akvaplan
[35.198.160.181] out: Scanning dependencies of target fabm
[35.198.160.181] out: [ 94%] Building Fortran object CMakeFiles/fabm.dir/fabm_coupling.F90.o
[35.198.160.181] out: [ 95%] Building Fortran object CMakeFiles/fabm.dir/fabm_library.F90.o
[35.198.160.181] out: [ 96%] Building Fortran object CMake

[35.198.160.181] out: [ 21%] Built target fabm_models_msi
[35.198.160.181] out: Scanning dependencies of target fabm_models_au
[35.198.160.181] out: [ 22%] Building Fortran object fabm_src/models/au/CMakeFiles/fabm_models_au.dir/prey_predator/jacob_monod.F90.o
[35.198.160.181] out: [ 22%] Building Fortran object fabm_src/models/au/CMakeFiles/fabm_models_au.dir/prey_predator/lotka_volterra.F90.o
[35.198.160.181] out: [ 23%] Building Fortran object fabm_src/models/au/CMakeFiles/fabm_models_au.dir/prey_predator/prey_predator.F90.o
[35.198.160.181] out: [ 23%] Building Fortran object fabm_src/models/au/CMakeFiles/fabm_models_au.dir/model_library.F90.o
[35.198.160.181] out: [ 23%] Built target fabm_models_au
[35.198.160.181] out: Scanning dependencies of target fabm_models_bb
[35.198.160.181] out: [ 24%] Building Fortran object fabm_src/models/bb/CMakeFiles/fabm_models_bb.dir/filter_feeder/filter_feeder.F90.o
[35.198.160.181] out: [ 24%] Building Fortran object fabm_src/models/bb/CMakeFiles

[35.198.160.181] out: [ 44%] Building Fortran object fabm_src/CMakeFiles/fabm.dir/fabm_library.F90.o
[35.198.160.181] out: [ 45%] Building Fortran object fabm_src/CMakeFiles/fabm.dir/fabm.F90.o
[35.198.160.181] out: [ 45%] Building Fortran object fabm_src/CMakeFiles/fabm.dir/fabm_config.F90.o
[35.198.160.181] out: [ 46%] Building Fortran object fabm_src/CMakeFiles/fabm.dir/fabm_version.F90.o
[35.198.160.181] out: [ 47%] Linking Fortran static library libfabm.a
[35.198.160.181] out: [ 47%] Built target fabm
[35.198.160.181] out: Scanning dependencies of target version
[35.198.160.181] out: [ 48%] Retrieving description of last GOTM commit...
[35.198.160.181] out: -- On branch lake with commit id v5.3-400-g90cc2710
[35.198.160.181] out: [ 48%] Built target version
[35.198.160.181] out: Scanning dependencies of target util
[35.198.160.181] out: [ 48%] Building Fortran object CMakeFiles/util.dir/util/util.F90.o
[35.198.160.181] out: [ 48%] Building Fortran object CMakeFiles/util.dir/util/e

[35.198.160.181] out: [ 80%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/turbulence.F90.o
[35.198.160.181] out: [ 81%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/kpp.F90.o
[35.198.160.181] out: [ 82%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/algebraiclength.F90.o
[35.198.160.181] out: [ 82%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/alpha_mnb.F90.o
[35.198.160.181] out: [ 83%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/cmue_a.F90.o
[35.198.160.181] out: [ 83%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/cmue_b.F90.o
[35.198.160.181] out: [ 84%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/cmue_c.F90.o
[35.198.160.181] out: [ 84%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/cmue_d.F90.o
[35.198.160.181] out: [ 85%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/cmue_ma.F90.o
[35.198.160.181] out: [ 85%] Building Fortran object CMakeF

[35.198.160.181] out: Receiving objects:   5% (104/2070)   
[35.198.160.181] out: Receiving objects:   6% (125/2070)   
[35.198.160.181] out: Receiving objects:   7% (145/2070)   
[35.198.160.181] out: Receiving objects:   8% (166/2070)   
[35.198.160.181] out: Receiving objects:   9% (187/2070)   
[35.198.160.181] out: Receiving objects:  10% (207/2070)   
[35.198.160.181] out: Receiving objects:  11% (228/2070)   
[35.198.160.181] out: Receiving objects:  12% (249/2070)   
[35.198.160.181] out: Receiving objects:  13% (270/2070)   
[35.198.160.181] out: Receiving objects:  14% (290/2070)   
[35.198.160.181] out: Receiving objects:  15% (311/2070)   
[35.198.160.181] out: Receiving objects:  15% (315/2070), 1.21 MiB | 1.18 MiB/s   
[35.198.160.181] out: Receiving objects:  15% (319/2070), 5.96 MiB | 2.92 MiB/s   
[35.198.160.181] out: Receiving objects:  16% (332/2070), 5.96 MiB | 2.92 MiB/s   
[35.198.160.181] out: Receiving objects:  17% (352/2070), 12.06 MiB | 4.72 MiB/s   
[35.198

[35.198.160.181] out: Receiving objects:  98% (2029/2070), 197.12 MiB | 25.72 MiB/s   
[35.198.160.181] out: Receiving objects:  99% (2050/2070), 197.12 MiB | 25.72 MiB/s   
[35.198.160.181] out: Receiving objects: 100% (2070/2070), 197.12 MiB | 25.72 MiB/s   
[35.198.160.181] out: Receiving objects: 100% (2070/2070), 202.81 MiB | 25.13 MiB/s, done.
[35.198.160.181] out: remote: Total 2070 (delta 24), reused 0 (delta 0)
[35.198.160.181] out: Resolving deltas:   0% (0/1090)   
[35.198.160.181] out: Resolving deltas:   1% (17/1090)   
[35.198.160.181] out: Resolving deltas:   2% (29/1090)   
[35.198.160.181] out: Resolving deltas:   3% (37/1090)   
[35.198.160.181] out: Resolving deltas:   4% (49/1090)   
[35.198.160.181] out: Resolving deltas:  10% (113/1090)   
[35.198.160.181] out: Resolving deltas:  11% (126/1090)   
[35.198.160.181] out: Resolving deltas:  12% (131/1090)   
[35.198.160.181] out: Resolving deltas:  15% (167/1090)   
[35.198.160.181] out: Resolving deltas:  17% (187/1

[35.198.160.181] out: Checking out files:  84% (658/783)   
[35.198.160.181] out: Checking out files:  85% (666/783)   
[35.198.160.181] out: Checking out files:  86% (674/783)   
[35.198.160.181] out: Checking out files:  87% (682/783)   
[35.198.160.181] out: Checking out files:  88% (690/783)   
[35.198.160.181] out: Checking out files:  89% (697/783)   
[35.198.160.181] out: Checking out files:  90% (705/783)   
[35.198.160.181] out: Checking out files:  91% (713/783)   
[35.198.160.181] out: Checking out files:  92% (721/783)   
[35.198.160.181] out: Checking out files:  93% (729/783)   
[35.198.160.181] out: Checking out files:  94% (737/783)   
[35.198.160.181] out: Checking out files:  95% (744/783)   
[35.198.160.181] out: Checking out files:  96% (752/783)   
[35.198.160.181] out: Checking out files:  97% (760/783)   
[35.198.160.181] out: Checking out files:  98% (768/783)   
[35.198.160.181] out: Checking out files:  99% (776/783)   
[35.198.160.181] out: Checking out files